<a href="https://colab.research.google.com/github/uujeongLee/FASCODE/blob/main/task1_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://codegongbang.tistory.com/16

**안녕하세요? 이 코드는 도도돋도도 ....!
 데이터 전처리 및 모델까지 완료했어요.위에 있는 셀 N개만 돌리면 제출 가능합니다.!**

## 이미지 데이터를 파이썬 데이터로 변환하기

In [ ]:
!pip install -U aifactory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=ea238c3946cd171d3e5ab61e062f85f7564667f01d0d441ac111ac2e1bd5f145
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

%matplotlib inline

In [ ]:
# 이것은 전역 선언
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/ETRI_RE')  # my_project 디렉토리의 경로로 변경해야 합니다.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def CODICODING() :
  # 우리 패키지
  from PIL import Image
  import os, glob
  import numpy as np
  from sklearn.model_selection import train_test_split
  import csv
  import pandas as pd
  import matplotlib.pyplot as plt
  from keras.models import Sequential
  from keras.layers import Convolution2D, MaxPooling2D
  from keras.layers import Activation, Dropout, Flatten, Dense



  # ETRI 패키지

  from dataset import ETRIDataset_emo
  from networks import ResExtractor, Baseline_ResNet_emo

  import pandas as pd
  import numpy as np
  from sklearn.metrics import confusion_matrix

  import torch
  import torch.utils.data
  import torch.utils.data.distributed

  from tqdm import tqdm

  # Test-set 만들기
  Test_csv = pd.read_csv('/aif/Dataset/info_etri20_emotion_test.csv')
  # 분류 대상 카테고리 선택하기 --- (※1)
  test_dir = ETRIDataset_emo(Test_csv, base_path='/aif/Dataset/Test/')
  #nb_classes = len(categories)
  test_Y_Gender = Test_csv['Gender']
  test_Y_Daily = Test_csv['Daily']
  test_Y_Embellishment = Test_csv['Embellishment']

  # 이미지 크기 지정 --- (※2)
  image_w = 64
  image_h = 64
  pixels = image_w * image_h * 3

  # 이미지 데이터 읽어 들이기 --- (※3)
  test_X = []
  #Y = []
  for idx, image_name in enumerate(Test_csv['image_name']):
      #print(image_name)
      #print(idx)
      f = test_dir + '/' + image_name
      img = Image.open(f) # --- (※6)

      # x_min, y_min, x_max, y_max 정의
      x_min = int(Test_csv['BBox_xmin'][idx])
      y_min = int(Test_csv['BBox_ymin'][idx])
      x_max = int(Test_csv['BBox_xmax'][idx])
      y_max = int(Test_csv['BBox_ymax'][idx])

      # 좌표로 이미지 자르기
      crop_img = img.crop((x_min, y_min, x_max, y_max))

      img = crop_img.convert("RGB")
      img = img.resize((image_w, image_h))
      data = np.asarray(img)
      test_X.append(data)

  # 각각의 데이터들 numpy array로 변환
  test_X = np.array(test_X)
  test_Y_Gender = np.array(test_Y_Gender)
  test_Y_Daily = np.array(test_Y_Daily)
  test_Y_Embellishment = np.array(test_Y_Embellishment)

  # 정규화
  test_X  = test_X.astype("float")  / 256

  # 이미지 크기 지정하기
  image_w = 64
  image_h = 64


  # Gender 성
  # Gender_xy = (train_X, test_X, train_Y_Gender, test_Y_Gender)

  # 카테고리 지정하기
  Gender_categories = [0,1,2,3,4]
  Gender_nb_classes = len(Gender_categories)

  # Daily 일상성
  # 카테고리 지정하기
  Daily_categories = [0,1,2,3,4,5,6]
  Daily_nb_classes = len(Daily_categories)

  #Embellishment 장식성
  # 카테고리 지정하기
  Embellishment_categories = [0,1,2,3]
  nb_classes = len(Embellishment_categories)

  # Gender
  # Category 0~4를 2진수 배열 형태의 a~e로 변환
  # 5, 7, 4
  train_Y_Gender_bin = []
  test_Y_Gender_bin = []

  a = [1, 0, 0, 0, 0]
  b = [0, 1, 0, 0, 0]
  c = [0, 0, 1, 0, 0]
  d = [0, 0, 0, 1, 0]
  e = [0, 0, 0, 0, 1]

  for before ,name in [[test_Y_Gender,test_Y_Gender_bin] ] :
    for y in before :
      if y == 0:
          name.append(a)
      if y == 1:
          name.append(b)
      if y == 2:
          name.append(c)
      if y == 3:
          name.append(d)
      if y == 4:
          name.append(e)
  # Daily
  # Category 0~6를 2진수 배열 형태의 a~g로 변환

  train_Y_Daily_bin = []
  test_Y_Daily_bin = []

  a = [1, 0, 0, 0, 0, 0, 0]
  b = [0, 1, 0, 0, 0, 0, 0]
  c = [0, 0, 1, 0, 0, 0, 0]
  d = [0, 0, 0, 1, 0, 0, 0]
  e = [0, 0, 0, 0, 1, 0, 0]
  f = [0, 0, 0, 1, 0, 1, 0]
  g = [0, 0, 0, 0, 0, 0, 1]

  for before ,name in [[test_Y_Daily, test_Y_Daily_bin] ]  :
    for y in before :
      if y == 0:
          name.append(a)
      if y == 1:
          name.append(b)
      if y == 2:
          name.append(c)
      if y == 3:
          name.append(d)
      if y == 4:
          name.append(e)
      if y == 5:
          name.append(f)
      if y == 6:
          name.append(g)

  # Embellishment
  # Category 0~3를 2진수 배열 형태의 a~d로 변환

  train_Y_Embellishment_bin = []
  test_Y_Embellishment_bin = []

  a = [1, 0, 0, 0]
  b = [0, 1, 0, 0]
  c = [0, 0, 1, 0]
  d = [0, 0, 0, 1]


  for before ,name in [[test_Y_Embellishment,test_Y_Embellishment_bin] ] :
    for y in before :
      if y == 0:
          name.append(a)
      if y == 1:
          name.append(b)
      if y == 2:
          name.append(c)
      if y == 3:
          name.append(d)

  # 각 리스트들을 numpy array로 변환
  test_Y_Gender_bin = np.array(test_Y_Gender_bin)
  test_Y_Daily_bin = np.array(test_Y_Daily_bin)
  test_Y_Embellishment_bin = np.array(test_Y_Embellishment_bin)

  # 모델 불러오기
  from keras.models import load_model
  Gender_model = load_model("/content/drive/MyDrive/ETRI_RE/CNN_models/Gender_model_CNN.h5")          # 모델 불러오는 경로를 설정하거나 JUPYTER 에서 같은 디렉토리에 두어야할 것 같아요.
  Daily_model = load_model("/content/drive/MyDrive/ETRI_RE/CNN_models/Daily_model_CNN.h5")
  Embellishment_model = load_model("/content/drive/MyDrive/ETRI_RE/CNN_models/Embellishment_model_CNN.h5")

  #pred l 모델 명 각각 바꿔서 복사하기
  Gender_pred = Gender_model.predict(test_X)
  Gender_pred = np.argmax(pred, axis=1)
  Gender_pred = Gender_pred.astype(float)
  Gender_pred = np.round(Gender_pred, decimals =1 )

  #pred l 모델 명 각각 바꿔서 복사하기
  Daily_pred = Daily_model.predict(test_X)
  Daily_pred = np.argmax(pred, axis=1)
  Daily_pred  = Daily_pred.astype(float)
  Daily_pred  = np.round(Daily_pred , decimals =1 )

  #pred l 모델 명 각각 바꿔서 복사하기
  Embellishment_pred = Embellishment_model.predict(test_X)
  Embellishment_pred = np.argmax(Embellishment_pred, axis=1)
  Embellishment_pred = Embellishment_pred.astype(float)
  Embellishment_pred = np.round(Embellishment_pred, decimals =1 )

  out = pd.DataFrame({'image_name':df['image_name'], 'daily':Daily_pred, 'gender':Gender_pred, 'embel':Embellishment_pred})
  return out

In [ ]:
# 한번  Warpping
def submit() :
  return CODICODING

In [ ]:
import aifactory.score as aif
import time
t = time.time()
if __name__ == "__main__":
    #-----------------------------------------------------#
    aif.submit(model_name="코디코딩TASK1",                                        # 본인의 모델명 입력(버전 관리에 용이하게끔 편의에 맞게 지정합니다)
               key="b4c94d1c-c866-494e-b6bc-6c2d2e625685",                      # 본인의 task key 입력, 주석 해제하면 제출할 수 있어요. 될지는 모르겠습니다만,,,,
               func=submit                                                      # 3.에서 wrapping한 submit function
               )
    #-----------------------------------------------------#
    print(time.time() - t)

file : fileId=1pRnWxjLEuseoEC4XrsnURlfkdJDsR9bR
Running on CoLab
google colab


Downloading...
From: https://drive.google.com/uc?id=1pRnWxjLEuseoEC4XrsnURlfkdJDsR9bR
To: /content/task.ipynb
100%|██████████| 48.3k/48.3k [00:00<00:00, 28.2MB/s]


파일 전송 완료
error = [NbConvertApp] Converting notebook /aif/src/task.ipynb to notebook
2023-09-14 12:16:22.110802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 12:16:23.546974: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-09-14 12:16:23.547200: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 

**아래 코드부터는 돌리지 않아도 제출 가능합니다.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Train_csv = pd.read_csv("/content/drive/MyDrive/ETRI_RE/Dataset/info_etri20_emotion_train.csv")
Test_csv = pd.read_csv("/content/drive/MyDrive/ETRI_RE/Dataset/info_etri20_emotion_test_sample.csv")
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 행을 출력한다.

In [ ]:
# 분류 대상 카테고리 선택하기 --- (※1)
train_dir = "/content/drive/MyDrive/ETRI_RE/Dataset/Train"
train_Y_Gender = Train_csv['Gender']
train_Y_Daily = Train_csv['Daily']
train_Y_Embellishment = Train_csv['Embellishment']

# 이미지 크기 지정 --- (※2)
image_w = 64
image_h = 64
pixels = image_w * image_h * 3

# 이미지 데이터 읽어 들이기 --- (※3)
train_X = []
#Y = []
for idx, image_name in enumerate(Train_csv['image_name']):
    #print(image_name)
    print(idx, end=", ")
    f = train_dir + '/' + image_name
    img = Image.open(f) # --- (※6)

    # x_min, y_min, x_max, y_max 정의
    x_min = int(Train_csv['BBox_xmin'][idx])
    y_min = int(Train_csv['BBox_ymin'][idx])
    x_max = int(Train_csv['BBox_xmax'][idx])
    y_max = int(Train_csv['BBox_ymax'][idx])

    # 좌표로 이미지 자르기
    crop_img = img.crop((x_min, y_min, x_max, y_max))

    img = crop_img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    train_X.append(data)


train_X = np.array(train_X)
train_Y_Gender = np.array(train_Y_Gender)
train_Y_Daily = np.array(train_Y_Daily)
train_Y_Embellishment = np.array(train_Y_Embellishment)


In [ ]:
# Test-set 만들기

# 분류 대상 카테고리 선택하기 --- (※1)
#test_dir = ETRIDataset_emo(Test_csv, base_path='/aif/Dataset/Test/')
test_dir = "/content/drive/MyDrive/ETRI_RE/Dataset/Test_sample"

test_Y_Gender = Test_csv['Gender']
test_Y_Daily = Test_csv['Daily']
test_Y_Embellishment = Test_csv['Embellishment']

# 이미지 크기 지정 --- (※2)
image_w = 64
image_h = 64
pixels = image_w * image_h * 3

# 이미지 데이터 읽어 들이기 --- (※3)
test_X = []
#Y = []
for idx, image_name in enumerate(Test_csv['image_name']):
    #print(image_name)
    #print(idx)
    f = test_dir + '/' + image_name
    img = Image.open(f) # --- (※6)

    # x_min, y_min, x_max, y_max 정의
    x_min = int(Test_csv['BBox_xmin'][idx])
    y_min = int(Test_csv['BBox_ymin'][idx])
    x_max = int(Test_csv['BBox_xmax'][idx])
    y_max = int(Test_csv['BBox_ymax'][idx])

    # 좌표로 이미지 자르기
    crop_img = img.crop((x_min, y_min, x_max, y_max))

    img = crop_img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    test_X.append(data)

# 각각의 데이터들 numpy array로 변환
test_X = np.array(test_X)
test_Y_Gender = np.array(test_Y_Gender)
test_Y_Daily = np.array(test_Y_Daily)
test_Y_Embellishment = np.array(test_Y_Embellishment)

In [ ]:
Gender_xy = (train_X, test_X, train_Y_Gender, test_Y_Gender)
Daily_xy = (train_X, test_X, train_Y_Daily, test_Y_Daily)
Embellishment_xy = (train_X, test_X, train_Y_Embellishment, test_Y_Embellishment)

# drive에 crop한 이미지 npy 형태로 저장
np.save("/content/drive/MyDrive/ETRI_RE/npy/Gender_xy.npy", Gender_xy)
np.save("/content/drive/MyDrive/ETRI_RE/npy/Daily_xy.npy", Daily_xy)
np.save("/content/drive/MyDrive/ETRI_RE/npy/Embellishment_xy.npy", Embellishment_xy)

## CNN 으로 분류하기 (케라스 이용)

In [ ]:
# 전처리 완료한 데이터 로드
train_X, test_X, train_Y_Gender, test_Y_Gender = np.load("/content/drive/MyDrive/ETRI_RE/Gender_xy.npy", allow_pickle=True)
train_X, test_X, train_Y_Daily, test_Y_Daily = np.load("/content/drive/MyDrive/ETRI_RE/Daily_xy.npy", allow_pickle=True)
train_X, test_X, train_Y_Embellishment, test_Y_Embellishment = np.load("/content/drive/MyDrive/ETRI_RE/Embellishment_xy.npy", allow_pickle=True)

# 데이터 정규화하기
train_X = train_X.astype("float") / 256
test_X  = test_X.astype("float")  / 256
print('train_X shape:', train_X.shape)

# 이미지 크기 지정하기
image_w = 64
image_h = 64

In [ ]:
# Gender 성
# Gender_xy = (train_X, test_X, train_Y_Gender, test_Y_Gender)

# 카테고리 지정하기
Gender_categories = [0,1,2,3,4]
Gender_nb_classes = len(Gender_categories)


In [ ]:
# Daily 일상성

# 카테고리 지정하기
Daily_categories = [0,1,2,3,4,5,6]
Daily_nb_classes = len(Daily_categories)


In [ ]:
#Embellishment 장식성

# 카테고리 지정하기
Embellishment_categories = [0,1,2,3]
nb_classes = len(Embellishment_categories)


In [ ]:
# Gender
# Category 0~4를 2진수 배열 형태의 a~e로 변환
# 5, 7, 4
train_Y_Gender_bin = []
test_Y_Gender_bin = []

a = [1, 0, 0, 0, 0]
b = [0, 1, 0, 0, 0]
c = [0, 0, 1, 0, 0]
d = [0, 0, 0, 1, 0]
e = [0, 0, 0, 0, 1]

for before ,name in [[train_Y_Gender, train_Y_Gender_bin],[test_Y_Gender,test_Y_Gender_bin] ] :
  for y in before :
    if y == 0:
        name.append(a)
    if y == 1:
        name.append(b)
    if y == 2:
        name.append(c)
    if y == 3:
        name.append(d)
    if y == 4:
        name.append(e)
train_Y_Gender_bin

In [ ]:
# Daily
# Category 0~6를 2진수 배열 형태의 a~g로 변환

train_Y_Daily_bin = []
test_Y_Daily_bin = []

a = [1, 0, 0, 0, 0, 0, 0]
b = [0, 1, 0, 0, 0, 0, 0]
c = [0, 0, 1, 0, 0, 0, 0]
d = [0, 0, 0, 1, 0, 0, 0]
e = [0, 0, 0, 0, 1, 0, 0]
f = [0, 0, 0, 1, 0, 1, 0]
g = [0, 0, 0, 0, 0, 0, 1]

for before ,name in [[train_Y_Daily , train_Y_Daily_bin], [test_Y_Daily, test_Y_Daily_bin] ]  :
  for y in before :
    if y == 0:
        name.append(a)
    if y == 1:
        name.append(b)
    if y == 2:
        name.append(c)
    if y == 3:
        name.append(d)
    if y == 4:
        name.append(e)
    if y == 5:
        name.append(f)
    if y == 6:
        name.append(g)
test_Y_Daily_bin

In [ ]:
# Embellishment
# Category 0~3를 2진수 배열 형태의 a~d로 변환

train_Y_Embellishment_bin = []
test_Y_Embellishment_bin = []

a = [1, 0, 0, 0]
b = [0, 1, 0, 0]
c = [0, 0, 1, 0]
d = [0, 0, 0, 1]


for before ,name in [[train_Y_Embellishment, train_Y_Embellishment_bin], [test_Y_Embellishment,test_Y_Embellishment_bin] ] :
  for y in before :
    if y == 0:
        name.append(a)
    if y == 1:
        name.append(b)
    if y == 2:
        name.append(c)
    if y == 3:
        name.append(d)
test_Y_Embellishment_bin

In [ ]:
# 각 리스트들을 numpy array로 변환
train_Y_Gender_bin = np.array(train_Y_Gender_bin)
test_Y_Gender_bin = np.array(test_Y_Gender_bin)

train_Y_Daily_bin = np.array(train_Y_Daily_bin)
test_Y_Daily_bin = np.array(test_Y_Daily_bin)

train_Y_Embellishment_bin = np.array(train_Y_Embellishment_bin)
test_Y_Embellishment_bin = np.array(test_Y_Embellishment_bin)

 **Gender classification 모델**

In [ ]:
# 모델 구축하기 --- (※2)
Gender_model = Sequential()
Gender_model.add(Convolution2D(32, (3, 3),
    padding='same',
    input_shape=train_X.shape[1:]))
Gender_model.add(Activation('relu'))
Gender_model.add(MaxPooling2D(pool_size=(2, 2)))
Gender_model.add(Dropout(0.25))

Gender_model.add(Convolution2D(64, (3, 3), padding='same'))
Gender_model.add(Activation('relu'))
Gender_model.add(Convolution2D(64, (3, 3)))
Gender_model.add(MaxPooling2D(pool_size=(2, 2)))
Gender_model.add(Dropout(0.25))

Gender_model.add(Flatten()) # --- (※3)
Gender_model.add(Dense(512))
Gender_model.add(Activation('relu'))
Gender_model.add(Dropout(0.5))
Gender_model.add(Dense(5))  #Dense(nb_classes) 이거 Y class 갯수만큼 바꿔줘야함. ex. 0~6 --> 7
Gender_model.add(Activation('softmax'))

Gender_model.compile(loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

In [ ]:
epoch = 50

In [ ]:
# 모델 훈련하기 --- (※4)
epoch = 50
Gender_model.fit(train_X, train_Y_Gender_bin, batch_size=32, epochs = epoch)

In [ ]:
# 모델 평가하기--- (※5)
Gender_score = Gender_model.evaluate(test_X, test_Y_Gender_bin)
print('loss=', Gender_score[0])
print('accuracy=', Gender_score[1])

In [ ]:
#os.mkdir('/content/drive/MyDrive/ETRI_RE/CNN_models')

In [ ]:
# 모델 저장하기
Gender_model.save("/content/drive/MyDrive/ETRI_RE/CNN_models/Gender_model_CNN.h5")

**Daily Clssification 모델**

In [ ]:
# 모델 구축하기 --- (※2)
Daily_model = Sequential()
Daily_model.add(Convolution2D(32, (3, 3),
    padding='same',
    input_shape=train_X.shape[1:]))
Daily_model.add(Activation('relu'))
Daily_model.add(MaxPooling2D(pool_size=(2, 2)))
Daily_model.add(Dropout(0.25))

Daily_model.add(Convolution2D(64, (3, 3), padding='same'))
Daily_model.add(Activation('relu'))
Daily_model.add(Convolution2D(64, (3, 3)))
Daily_model.add(MaxPooling2D(pool_size=(2, 2)))
Daily_model.add(Dropout(0.25))

Daily_model.add(Flatten()) # --- (※3)
Daily_model.add(Dense(512))
Daily_model.add(Activation('relu'))
Daily_model.add(Dropout(0.5))
Daily_model.add(Dense(7))  #Dense(nb_classes) 이거 Y class 갯수만큼 바꿔줘야함. ex. 0~6 --> 7
Daily_model.add(Activation('softmax'))

Daily_model.compile(loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

In [ ]:
# 모델 훈련하기 --- (※4)
epoch = 50
Daily_model.fit(train_X, train_Y_Daily_bin, batch_size=32, epochs = epoch)

In [ ]:
# 모델 평가하기--- (※5)
Daily_score = Daily_model.evaluate(test_X, test_Y_Daily_bin)
print('loss=', Daily_score[0])
print('accuracy=', Daily_score[1])


In [ ]:
# Train set 모델 평가하기--- (※5)
Daily_score = Daily_model.evaluate(train_X, train_Y_Daily_bin)
print('loss=', Daily_score[0])
print('accuracy=', Daily_score[1])

In [ ]:
# 모델 저장하기
Daily_model.save("/content/drive/MyDrive/ETRI_RE/CNN_models/Daily_model_CNN.h5")

**Embellishment Classification 모델**

In [ ]:
# 모델 구축하기 --- (※2)
Embellishment_model = Sequential()
Embellishment_model.add(Convolution2D(32, (3, 3),
    padding='same',
    input_shape=train_X.shape[1:]))
Embellishment_model.add(Activation('relu'))
Embellishment_model.add(MaxPooling2D(pool_size=(2, 2)))
Embellishment_model.add(Dropout(0.25))

Embellishment_model.add(Convolution2D(64, (3, 3), padding='same'))
Embellishment_model.add(Activation('relu'))
Embellishment_model.add(Convolution2D(64, (3, 3)))
Embellishment_model.add(MaxPooling2D(pool_size=(2, 2)))
Embellishment_model.add(Dropout(0.25))

Embellishment_model.add(Flatten()) # --- (※3)
Embellishment_model.add(Dense(512))
Embellishment_model.add(Activation('relu'))
Embellishment_model.add(Dropout(0.5))
Embellishment_model.add(Dense(4))  #Dense(nb_classes) 이거 Y class 갯수만큼 바꿔줘야함. ex. 0~6 --> 7
Embellishment_model.add(Activation('softmax'))

Embellishment_model.compile(loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

In [ ]:
# 모델 훈련하기 --- (※4)
epoch = 50
Embellishment_model.fit(train_X, train_Y_Embellishment_bin, batch_size=32, epochs = epoch)

In [ ]:
# 모델 평가하기--- (※5)
Embellishment_score = Embellishment_model.evaluate(test_X, test_Y_Embellishment_bin)
print('loss=', Embellishment_score[0])
print('accuracy=', Embellishment_score[1])

In [ ]:
# 모델 저장하기
Embellishment_model.save("/content/drive/MyDrive/ETRI_RE/CNN_models/Embellishment_model_CNN.h5")

In [ ]:
# 원본 코드
# 모델 구축하기 --- (※2)
model = Sequential()
model.add(Convolution2D(32, (3, 3),
    padding='same',
    input_shape=train_X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) # --- (※3)
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))  #Dense(nb_classes) 이거 Y class 갯수만큼 바꿔줘야함. ex. 0~6 --> 7
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

# 모델 훈련하기 --- (※4)
epoch = 30
model.fit(train_X, train_Y_Gender_bin, batch_size=32, epochs = epoch)

# 모델 평가하기--- (※5)
score = model.evaluate(X_test, y_test_re)
print('loss=', score[0])
print('accuracy=', score[1])

# 모델 저장하기
model.save("CNN_cutted_gender.h5")


In [ ]:
# 모델 훈련하기 --- (※4)
model.fit(X_train, y_train_re, batch_size=32, nb_epoch=50)

# 모델 평가하기--- (※5)
score = model.evaluate(X_test, y_test_re)
print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
# 모델 평가하기--- (※5)
score = model.evaluate(X_test, y_test_re)
print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
# 모델 평가하기--- TRAIN SET
score = model.evaluate(X_train, y_train_re)
print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
# 모델 저장하기
model.save("CNN_cutted_gender.h5")

In [ ]:
# 모델 불러오기
from keras.models import load_model
model_copy = load_model("CNN_gender.h5")
model_copy.summary()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Activation

# 모델 구축하기
model = Sequential()
model.add(Convolution2D(32, 3, 3,
    border_mode='same',
    input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))  # Output node is changed to 1 for binary classification
model.add(Activation('sigmoid'))  # Use sigmoid activation for binary classification

model.compile(loss='binary_crossentropy',  # Use 'binary_crossentropy' for binary classification
    optimizer='rmsprop',
    metrics=['accuracy'])

# 모델 훈련하기
model.fit(X_train, y_train, batch_size=32, epochs=50)  # 'nb_epoch' is replaced with 'epochs'

# 모델 평가하기
score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])